<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# Conectando o drive ao colab #

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Treinar novo Modelo !

## Baixando a CNN pré-treinada ##


In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

## Carregando os arquivos do drive no colab ##

In [0]:
!unzip "/content/drive/My Drive/Colab/datasets/barcode4.zip" 

## 5. Treinando o modelo ##

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=11, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

## 6. Avaliando os modelos gerados ##

In [0]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# Utilizar modelo já treinado.

## Criando as pastas ##

In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(dirs)
file_count_corridas = file_count_corridas - 1
print("Número de corridas na pasta: ", file_count_corridas)


newFolderCorrida = str(file_count_corridas)
folder = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida
!mkdir $folder


Número de corridas na pasta:  4
mkdir: cannot create directory ‘/content/drive/My Drive/Colab/Corridas/Corrida_4’: File exists


## 8. Setando as variaveis ##


In [0]:
modelPath = "/content/drive/My Drive/Colab/models/barcode/detection_model-ex-007--loss-0003.066.h5"
JsonPath = "drive/My Drive/Colab/json/detection_config_barcode.json"
InputImage = "drive/My Drive/Colab/Imagens_Originais/img1.jpg"

## 9. Treinando modelo e recortando os codigos de barras da imagem original ##


In [0]:
from imageai.Detection.Custom import CustomObjectDetection
import cv2

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

InputImage = cv2.imread(InputImage, cv2.IMREAD_GRAYSCALE)

detections, extracted_objects_array = detector.detectObjectsFromImage(input_image = InputImage, 
                                                                      output_image_path="barcode-detected.jpg",
                                                                      minimum_percentage_probability=30,
                                                                      extract_detected_objects=True)
def boxPoints(e):
  return e['box_points']

detections.sort(key=boxPoints)

print("Número       " , ": ", "Porcentagem ", "      :    ", "box_points")
print("----------------------------------------------------------------")

print("")

for detection, object_path in zip(detections, extracted_objects_array):
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])


print("----------------------------------------------------------------")
print("Quantidade de código de barras detectados na imagem: ", len(detections))

In [0]:
  def removeDuplicates():
      i=0
      soma=0
      for detection in detections:
        if( i != len(detections)-1 ):
            
            Xa = (detections[i]['box_points'][0] + detections[i]['box_points'][2])/2
            Ya = (detections[i]['box_points'][1] + detections[i]['box_points'][3])/2
            Xb = (detections[i+1]['box_points'][0] + detections[i+1]['box_points'][2])/2
            Yb = (detections[i+1]['box_points'][1] + detections[i+1]['box_points'][3])/2

            Dab = ( (Xb-Xa)**2 + (Yb - Ya)**2 )**1/2
            if(Dab < float(10)):
                if(detections[i]["percentage_probability"] < detections[i+1]["percentage_probability"]):
                    detections.pop(i)
                else:
                    detections.pop(i+1)

        soma = soma + detections[i]["percentage_probability"]
        i=i+1

## 10. Movendo as pastas para o diretório correto. ##


In [0]:
!mv /content/barcode-detected.jpg $folder
!mv /content/barcode-detected-objects $folder

barcode = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_images"
!mv  $barcode $images